In [ ]:
addprocs(14)
workers()

In [1]:
push!(LOAD_PATH, "/home/artur/BondPricing/Julia/modules/")
include(string(LOAD_PATH[end], "AnalyticFunctions.jl"))

AnalyticFunctions

In [ ]:
include(string(LOAD_PATH[end], "AnalyticFunctions.jl"))
reload("AnalyticFunctions")

In [2]:
pardict = Dict("V0" => 100, 
               "vbl" => 60, 
               "vbh" => 62, 
                "m" => 1.,
                "c" => 6.39,
                "p" => 61.68,
                "sigmal" => .25,
                "sigmah" => .25,
                "r" => .08,
                "gross_delta" => .02,
                "iota" => .0,
                "xi" => 1.,
                "kappa" => .015,
                "alpha" => .6,
                "pi" => .27,
                "lambda" => .3)

Dict{String,Any} with 16 entries:
  "c"           => 6.39
  "vbh"         => 62
  "sigmah"      => 0.25
  "r"           => 0.08
  "gross_delta" => 0.02
  "sigmal"      => 0.25
  "vbl"         => 60
  "kappa"       => 0.015
  "lambda"      => 0.3
  "xi"          => 1.0
  "alpha"       => 0.6
  "iota"        => 0.0
  "V0"          => 100
  "pi"          => 0.27
  "m"           => 1.0
  "p"           => 61.68

In [ ]:
?AnalyticFunctions.pvvb_bond_surf

In [3]:
using Parameters

# @with_kw mutable struct Firm
#     V0::Float16
#     c::Float16
#     p::Float16
#     vbl::Float32
#     vbh::Float32

#     pm::FirmParams

# end

@with_kw struct FirmParams
    m::Float16
    alpha::Float16
    pi::Float16
    
    r::Float16
    gross_delta::Float16
    iota::Float16

    xi::UInt8
    kappa::Float16
    
    lambda::Float16
    sigmal::Float16
    sigmah::Float16
    
#     fc::FirmChoices

end

@with_kw mutable struct Firm
    V0::Float16
    c::Float16
    p::Float16
    vbl::Float32
    vbh::Float32

    pm::FirmParams

end

Firm

In [5]:
# # Firm Constructor
# function firm_constructor(dict)
    
#     return Firm(dict["m"], dict["alpha"], dict["pi"],
#                 dict["r"], dict["gross_delta"], dict["iota"],
#                 dict["xi"], dict["kappa"],
#                 dict["lambda"], dict["sigmal"], dict["sigmah"],
#                 FirmChoices(dict["V0"], dict["c"], dict["p"],
#                    dict["vbl"], dict["vbh"]))
# end

function firm_constructor(dict)
    
    return Firm(dict["V0"], dict["c"], dict["p"],
                   dict["vbl"], dict["vbh"],
                FirmParams(dict["m"], dict["alpha"], dict["pi"],
                dict["r"], dict["gross_delta"], dict["iota"],
                dict["xi"], dict["kappa"],
                dict["lambda"], dict["sigmal"], dict["sigmah"]))
end


firm_constructor (generic function with 1 method)

In [6]:
arm = firm_constructor(pardict)

Firm
  V0: Float16 100.0
  c: Float16 6.39
  p: Float16 61.7
  vbl: Float32 60.0
  vbh: Float32 62.0
  pm: FirmParams


In [66]:
# addprocs(-19)
include(string(LOAD_PATH[end], "AnalyticFunctions.jl"))
reload("AnalyticFunctions")
# addprocs(-19)

In [104]:
tauN = 10^3
ttm = arm.pm.m
dt = (ttm - 1e-4)/tauN
ugrid = linspace(1e-4, ttm, tauN)

vmax = 1.5
vN = 10^3
dv = vmax/vN
vgrid = linspace(0, vmax, vN)

# vbh = AnalyticFunctions.zhi_vb(arm.pm.m, arm.c, arm.p, arm.pm.sigmah, 
#                                 arm.pm.r, arm.pm.gross_delta, arm.pm.iota, 
#                                 arm.pm.xi, arm.pm.kappa, arm.pm.alpha, arm.pm.pi)


# ########################################################
# ########################################################
# Need to integrate for values above vmax!
# ########################################################
# ########################################################

# #######
vbhlmin = .8
vbhlmax = 1.2
vbhlN = 5
vbhlgrid = linspace(vbhlmin, vbhlmax, vbhlN)

#vtmax = log(bondVmax/vbmin)
vtmax = .8
vtN = 20
vtgrid = linspace(0, vtmax, vtN)

tic()

resPsi, cubePsi, cubeFPsi, cubeGPsi = AnalyticFunctions.interpd_bondpr_funs(vtgrid, vbhlgrid,
                                                         ugrid, vgrid,
                                                         arm.pm.sigmal, arm.pm.sigmah,
                                                         arm.pm.r, arm.pm.gross_delta,
                                                         arm.pm.xi, arm.pm.kappa, arm.pm.lambda)

toc()

elapsed time: 208.073794605 seconds


208.073794605

In [137]:
# 1. Interpolate Functions for Debt Prices for Vt = V0
# 2. Test interpolated bond prices for vbl != vbh (use Python code to check)

55.46666666666667

In [115]:
[cubeGPsi_sitp[x, .5] for x=vbhlgrid_refined]

10000-element Array{Float64,1}:
 0.000399814
 0.000400183
 0.000400553
 0.000400923
 0.000401292
 0.000401662
 0.000402032
 0.000402401
 0.000402771
 0.000403141
 0.00040351 
 0.00040388 
 0.00040425 
 ⋮          
 0.0307461  
 0.0307523  
 0.0307585  
 0.0307647  
 0.0307709  
 0.0307772  
 0.0307834  
 0.0307896  
 0.0307958  
 0.030802   
 0.0308082  
 0.0308144  

In [105]:
using Interpolations

vbhlgrid_refined_size = 10^4
vbhlgrid_refined = linspace(vbhlgrid[1], vbhlgrid[end], vbhlgrid_refined_size)


vtgrid_refined_size = 10^4
vtgrid_refined = linspace(vtgrid[1], vtgrid[end], vtgrid_refined_size)

# Maybe pass a single vmax
tic()
resPsi_itp = interpolate(resPsi, BSpline(Cubic(Line())), OnGrid())
resPsi_sitp = Interpolations.scale(resPsi_itp, vtgrid)

cubePsi_itp = interpolate(cubePsi, BSpline(Cubic(Line())), OnGrid())
cubePsi_sitp = Interpolations.scale(cubePsi_itp, vtgrid) # Scale
# cubePsiref = [cubePsi_sitp[x] for x in vtgrid_refined]

cubeFPsi_itp = interpolate(cubeFPsi, BSpline(Cubic(Line())), OnGrid())
cubeFPsi_sitp = Interpolations.scale(cubeFPsi_itp, vbhlgrid, vtgrid) # Scale
# cubeFPsiref = [cubeFPsi_sitp[vbl, vt] for vbl=vtgrid_refined, vt=vtgrid_refined]

cubeGPsi_itp = interpolate(cubeGPsi, BSpline(Cubic(Line())), OnGrid())
cubeGPsi_sitp = Interpolations.scale(cubeGPsi_itp, vbhlgrid, vtgrid) # Scale
# cubeGPsiref = [cubeGPsi_sitp[vbl, vt] for vbl=vtgrid_refined, vt=vtgrid_refined]
toc()

elapsed time: 0.001308706 seconds


0.001308706

In [118]:
resPsi

20-element Array{Float64,1}:
 0.0        
 5.12722e-11
 1.59802e-10
 4.40745e-10
 1.17215e-9 
 3.03349e-9 
 7.64813e-9 
 1.87899e-8 
 4.49895e-8 
 1.04997e-7 
 2.38881e-7 
 5.29905e-7 
 1.1463e-6  
 2.41859e-6 
 4.97821e-6 
 9.99826e-6 
 1.95981e-5 
 3.75014e-5 
 7.00711e-5 
 0.000127883

In [110]:
cubeGPsi_sitp[0., .5]

-0.31253740659654217

In [128]:
include(string(LOAD_PATH[end], "AnalyticFunctions.jl"))
reload("AnalyticFunctions")


tic()
AnalyticFunctions.bond_pr_interp(arm, 90., vbh, 1., vmax,
                resPsi_sitp, cubePsi_sitp, cubeFPsi_sitp, cubeGPsi_sitp)
toc()

elapsed time: 0.147941015 seconds


0.147941015

In [126]:
tic()
AnalyticFunctions.bond_pr_interp(arm, 90., vbh, 1., vmax,
                resPsi_sitp, cubePsi_sitp, cubeFPsi_sitp, cubeGPsi_sitp)
toc()

13.168024650444888
7.841430460359037e-7
elapsed time: 0.000252969 seconds


0.000252969

In [130]:
tic()
bvec_Future = @spawn [AnalyticFunctions.bond_pr_interp(arm, Vt, vbh, 1., vmax,
                resPsi_sitp, cubePsi_sitp, cubeFPsi_sitp, cubeGPsi_sitp) for Vt=linspace(0, 100, 125)]
bvec = fetch(bvec_Future)
toc()

elapsed time: 0.054965438 seconds


0.054965438

In [134]:
ttm_grid_size

LoadError: [91mUndefVarError: ttm_grid_size not defined[39m

In [136]:
tic()
ttm_grid_size = 20
debt = AnalyticFunctions.get_debt_price(log(100/vbh), vbh, vbh, ttm_grid_size,
                            arm.pm.m, arm.c, arm.p,
                              arm.pm.r, arm.pm.gross_delta, arm.pm.iota,
                              arm.pm.xi, arm.pm.kappa, arm.pm.alpha, arm.pm.pi,
                              arm.pm.lambda, arm.pm.sigmal, arm.pm.sigmah)
toc()
debt

Bond Percentage Difference: 0.0007263913640629347
elapsed time: 42.45273372 seconds


61.2870286588927

In [120]:
AnalyticFunctions.bond_pr_main(log(90/vbh), 1., vbh, vbh,
                              arm.pm.m, arm.c, arm.p,
                              arm.pm.r, arm.pm.gross_delta, arm.pm.iota,
                              arm.pm.xi, arm.pm.kappa, arm.pm.alpha, arm.pm.pi,
                              arm.pm.lambda, arm.pm.sigmal, arm.pm.sigmah)#, vmax=NaN)

57.4796086573383

In [89]:
vbh= AnalyticFunctions.zhi_vb(arm.pm.m, arm.c, arm.p,
    arm.pm.sigmah, arm.pm.r, arm.pm.gross_delta, arm.pm.iota, arm.pm.xi, arm.pm.kappa, arm.pm.alpha, arm.pm.pi)

68.2967089427127

In [29]:
?AnalyticFunctions.rdisc

No documentation found.

`AnalyticFunctions.rdisc` is a `Function`.

```
# 1 method for generic function "rdisc":
rdisc(r, xi, k) in AnalyticFunctions at /home/artur/BondPricing/Julia/modules/AnalyticFunctions.jl:13
```


In [ ]:
function bond_pr_interp(st, vt, vbl, ttm, vmax,
                        resPsi, iPsi, iFPsi, iGPsi)
    
    # Capital Structure
    m = st.pm.m
    c = st.c
    p = st.p
    
    # Dynamics
    r = st.pm.r
    gross_delta = st.pm.gross_delta
    iota = st.pm.iota
    
    # Liquidity
    xi = st.pm.xi
    k = st.pm.kappa
    
    # Volatility
    _lambda = st.pm.lambda
    sigmal = st.pm.sigmal
    sigmah = st.pm.sigmah
    
    # Default Barrier 
    vbh = zhi_vb(m, c, p, sigma, r, gross_delta, iota, xi, k, alpha, pi)

    
    if vt < 0
            return NaN
    else
        # Maturity or Default prior to Volatility Shock:
        cf0 = no_vol_shock_cf_pv(vt, vb, ttm,
                                 m, c, p, sigmal,
                                 r, gross_delta,
                                 xi, k, alpha, _lambda)

        # Volatility Shock Prior to Maturity:
        cf1 = c/rdisc(r, xi, k) * Psi[v] + 
              (p - c/rdisc(r, xi, k)) * iFPsi[log(vbh/vbl), v] +
              (alpha * vbh/m - c/rdisc(r, xi, k)) * iGPsi[log(vbh/vbl), v]
        cf2 = rfbond_price(ttm, c, p, r, xi, k) * resPsi[ttm]

        return cf0 + cf1 + cf2
    end
end

In [60]:
?AnalyticFunctions.zhi_vb()

No documentation found.

`AnalyticFunctions.zhi_vb` is a `Function`.

```
# 1 method for generic function "zhi_vb":
zhi_vb(m, c, p, sigma, r, gross_delta, iota, xi, k, alpha, pi) in AnalyticFunctions at /home/artur/BondPricing/Julia/modules/AnalyticFunctions.jl:117
```


In [ ]:
u = .5
ttm = arm.pm.m
vmax=2
vbl = 55
vt = log(arm.V0/vbl)
v = .8
#vbh = AnalyticFunctions.zhi_vb(arm.rm.m, c, p, sigma, r, gross_delta, iota, xi, k, alpha, pi)
vbh = 60

# AnalyticFunctions.vol_shock_cf_integrand_FPsi(vt, v, vbl, vbh,
#                             u, ttm,
#                             arm.pm.sigmal, arm.pm.sigmah,
#                             arm.pm.r, arm.pm.gross_delta,
#                             arm.pm.xi, arm.pm.kappa, arm.pm.lambda)

tauN = 10^3
vN = 10^3
dt = (ttm - 1e-4)/tauN
dv = vmax/vN

ugrid = linspace(1e-4, ttm, tauN)
vgrid = linspace(0, vmax, vN)

# #######
vblmin = 50
vblmax = 65
vblN = 2
vblgrid = linspace(vblmin, vblmax, vblN)

#vtmax = log(bondVmax/vbmin)
vtmax = 1.2
vtN = 5
vtgrid = linspace(0, vtmax, vtN)


# #######
sigmal = arm.pm.sigmal
sigmah = arm.pm.sigmah
r = arm.pm.r
gross_delta = arm.pm.gross_delta
xi = arm.pm.xi 
kappa = arm.pm.kappa
_lambda = arm.pm.lambda


tic()
cubePsi_Future = @spawn [AnalyticFunctions.bond_int_Psi(vt,
                                ttm,
                                sigmal, 
                                r, gross_delta,
                                xi, kappa, _lambda,
                                ugrid, vgrid) * dt * dv for vt=vtgrid]
cubePsi = fetch(cubePsi_Future)

cubeFPsi_Future = @spawn [AnalyticFunctions.bond_int_FPsi(vt, vbl, vbh,
                                ttm, # vmax,
                                sigmal, sigmah,
                                r, gross_delta,
                                xi, kappa, _lambda,
                                ugrid, vgrid) * dt * dv for vbl=vblgrid, vt=vtgrid]
cubeFPsi = fetch(cubeFPsi_Future)

cubeGPsi_Future = @spawn [AnalyticFunctions.bond_int_GPsi(vt, vbl, vbh,
                                ttm, # vmax,
                                sigmal, sigmah,
                                r, gross_delta,
                                xi, kappa, _lambda,
                                ugrid, vgrid) * dt * dv for vbl=vblgrid, vt=vtgrid]
cubeGPsi = fetch(cubeGPsi_Future)

# Psi = AnalyticFunctions.bond_int_Psi(vt, v,
#                                 u, ttm, # vmax,
#                                 arm.pm.sigmal, 
#                                 arm.pm.r, arm.pm.gross_delta,
#                                 arm.pm.xi, arm.pm.kappa, arm.pm.lambda,
#                                 ugrid, vgrid) * dt * dv

# FPsi = AnalyticFunctions.bond_int_FPsi(vt, v, vbl, vbh,
#                                         u, ttm, # vmax,
#                                         arm.pm.sigmal, arm.pm.sigmah,
#                                         arm.pm.r, arm.pm.gross_delta,
#                                         arm.pm.xi, arm.pm.kappa, arm.pm.lambda,
#                                         ugrid, vgrid) * dt * dv

# GPsi= AnalyticFunctions.bond_int_GPsi(vt, v, vbl, vbh,
#                                         u, ttm, # vmax,
#                                         arm.pm.sigmal, arm.pm.sigmah,
#                                         arm.pm.r, arm.pm.gross_delta,
#                                         arm.pm.xi, arm.pm.kappa, arm.pm.lambda,
#                                         ugrid, vgrid) * dt * dv

   
toc()
cubePsi
cubeFPsi
cubeGPsi
# println(string("Psi: ", Psi))
# println(string("FPsi: ", FPsi))
# println(string("GPsi: ", GPsi))

In [ ]:
tauN = 10^3
vN = 10^3
dt = (ttm - 1e-4)/tauN
dv = vmax/vN

ugrid = linspace(1e-4, ttm, tauN)
vgrid = linspace(0, vmax, vN)

# #######
vblmin = 50
vblmax = 65
vblN = 2
vblgrid = linspace(vblmin, vblmax, vblN)

#vtmax = log(bondVmax/vbmin)
vtmax = 1.2
vtN = 5
vtgrid = linspace(0, vtmax, vtN)

tic()
a, b, c = AnalyticFunctions.interpolate_bondpr_funs(vtmax, vtN,
                                     vblmin, vblmax, vblN, vbh,
                                     sigmal, sigmah,
                                     r, gross_delta,
                                     xi, kappa, _lambda,
                                     ugrid, vgrid)

toc()

In [ ]:
a

In [ ]:
cubeGPsi

In [ ]:
using Interpolations

vblgrid_refined_size = 10^4
vblgrid_refined = linspace(vblgrid[1], vblgrid[end], vblgrid_refined_size)


vtgrid_refined_size = 10^4
vtgrid_refined = linspace(vtgrid[1], vtgrid[end], vtgrid_refined_size)

# Maybe pass a single vmax
tic()
cubePsi_itp = interpolate(cubePsi, BSpline(Cubic(Line())), OnGrid())
cubePsi_sitp = Interpolations.scale(cubePsi_itp, vtgrid) # Scale
# cubePsiref = [cubePsi_sitp[x] for x in vtgrid_refined]

cubeFPsi_itp = interpolate(cubeFPsi, BSpline(Cubic(Line())), OnGrid())
cubeFPsi_sitp = Interpolations.scale(cubeFPsi_itp, vblgrid, vtgrid) # Scale
# cubeFPsiref = [cubeFPsi_sitp[vbl, vt] for vbl=vtgrid_refined, vt=vtgrid_refined]

cubeGPsi_itp = interpolate(cubeFPsi, BSpline(Cubic(Line())), OnGrid())
cubeGPsi_sitp = Interpolations.scale(cubeGPsi_itp, vblgrid, vtgrid) # Scale
# cubeGPsiref = [cubeGPsi_sitp[vbl, vt] for vbl=vtgrid_refined, vt=vtgrid_refined]
toc()

In [ ]:
cubePsi

In [ ]:
cubeFPsi

In [ ]:
# include(string(LOAD_PATH[end], "AnalyticFunctions.jl"))
# reload("AnalyticFunctions")
tic()

u = .5
ttm = arm.pm.m
vmax=2
vbl = 55
vt = log(arm.V0/vbl)
v = .8
#vbh = AnalyticFunctions.zhi_vb(arm.rm.m, c, p, sigma, r, gross_delta, iota, xi, k, alpha, pi)
vbh = 60

# AnalyticFunctions.vol_shock_cf_integrand_FPsi(vt, v, vbl, vbh,
#                             u, ttm,
#                             arm.pm.sigmal, arm.pm.sigmah,
#                             arm.pm.r, arm.pm.gross_delta,
#                             arm.pm.xi, arm.pm.kappa, arm.pm.lambda)

# tauN = 10^3
# vN = 10^3
# dt = (ttm - 1e-4)/tauN
# dv = vmax/vN

# ugrid = linspace(1e-4, ttm, tauN)
# vgrid = linspace(0, vmax, vN)



Psi, FPsi, GPsi= AnalyticFunctions.bond_int_terms(vt, v, vbl, vbh,
                                        u, ttm, vmax,
                                        arm.pm.sigmal, arm.pm.sigmah,
                                        arm.pm.r, arm.pm.gross_delta,
                                        arm.pm.xi, arm.pm.kappa, arm.pm.lambda)

   
toc()

println(string("Psi: ", Psi))
println(string("FPsi: ", FPsi))
println(string("GPsi: ", GPsi))

In [ ]:

tic()
pmin = 50
pmax = 70
pN = 2
vN = 20
vbmin = 50
vbmax = 70
vbN = 2

cube = AnalyticFunctions.pvvb_bond_surf(arm.V0, pmin, pmax, pN, vN,
                            vbmin, vbmax, vbN,
                            arm.pm.m, arm.c, arm.p,
                            arm.pm.r, arm.pm.gross_delta, arm.pm.iota,
                            arm.pm.xi, arm.pm.kappa,
                            arm.pm.alpha, arm.pm.pi,
                            arm.pm.lambda, arm.pm.sigmal, arm.pm.sigmah)
toc()

In [ ]:
120/80 * 60000 / 3600

In [ ]:
cube

In [ ]:
@unpack (sigmal,
            sigmah) = arm.pm

In [ ]:
@unpack V0 = arm

In [ ]:
4 * (3 > 4)

In [ ]:
arm.V0

In [ ]:
sigmah

In [ ]:
@unpack V0 = arm

In [ ]:
V0

In [ ]:
arm.fc.V0 = 200

In [ ]:
tmp = afun.return_args(afun.zhi_vb)

In [ ]:
a = methods(afun.zhi_vb)

In [ ]:
methods(afun.zhi_vb).mt

In [ ]:
a.ms[1].unspecialized

In [ ]:
fieldnames(a.mt.defs.func)

In [ ]:
b

In [ ]:
fieldnames(a.ms[1])

In [ ]:
 ast = Base.uncompressed_ast(methods(afun.zhi_vb).defs.func.code)

In [ ]:
sigmah*sigmal

In [ ]:
function get_vb(a::Firm)
    @unpack m = a
    println(string(['m value is:  ', m]))
end

In [ ]:
?afun.zhi_vb